TASK 2 OF CREATING A RESTUARANT  RECOMMENDATION SYSTEM USING MACHINE LEARNING 

Let us load the required libraries first

In [5]:
import numpy as np 
import pandas as pd 
import difflib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

DATA COLLECTION AND PREPROCESSING 

Loading the required dataset for creating a recommendation system

In [6]:
df=pd.read_csv("C:/Users/aniru/Downloads/Dataset .csv")

In [7]:
df.head()

Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM Megamall, Ortigas, Mandaluyong City   
4      SM Megamall, Ortigas, Mandaluyong City   

                                    Locality Verbose   Longitude   Latitude  \
0  Century City Mall, Poblacion, Makati City, Mak...  121.027535  14.565443   
1  Little Tokyo, Legaspi Village, Makati City, Ma...  121.014101  14.553708   
2  Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...  121.056831  14.581404   
3  SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.056475  14.585318   
4  SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.057508  14.584450   

                           Cuisines  ...          Currency Has Table booking  \
0        French, Japanese, Desserts  ...  Botswana Pula(P)               Yes   
1                          Japanese  ...  Botswana Pula(P)               Yes   
2  Seafood, Asian, Filipino, Indian  ...  Botswana Pula(P)               Yes   
3                   Japanese, Sushi  ...  Botswana Pula(P)                No   
4                  Japanese, Korean  ...  Botswana Pula(P)               Yes   

  Has Online delivery Is delivering now Switch to order menu Price range  \
0                  No                No                   No           3   
1                  No                No                   No           3   
2                  No                No                   No           4   
3                  No                No                   No           4   
4                  No                No                   No           4   

   Aggregate rating  Rating color Rating text Votes  
0               4.8    Dark Green   Excellent   314  
1               4.5    Dark Green   Excellent   591  
2               4.4         Green   Very Good   270  
3               4.9    Dark Green   Excellent   365  
4               4.8    Dark Green   Excellent   229  

[5 rows x 21 columns]

In [8]:
df.shape

(9551, 21)

The dataset has 9551 rows and 21 columns . The rows represent the observations taken n each restuarant and the columns represent the features of the restuarants on which the observartions are taken

We now select the relevant features for recommendations . Our task is to create a restuarant recommendation system based on users preferences 

In [9]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

checking for any null values in the data set 

In [10]:
df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

We see there are no null values as such

Checking for duplicate values in the system

In [11]:
df[df.duplicated]

Empty DataFrame
Columns: [Restaurant ID, Restaurant Name, Country Code, City, Address, Locality, Locality Verbose, Longitude, Latitude, Cuisines, Average Cost for two, Currency, Has Table booking, Has Online delivery, Is delivering now, Switch to order menu, Price range, Aggregate rating, Rating color, Rating text, Votes]
Index: []

[0 rows x 21 columns]

There are no duplicate rows as such

We shall now encode the categorical variables 

We now combine all the selected features 


In [12]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

Feature Extraction

In [13]:
Selected_features_table=df.drop(columns=["Rating color","Rating text","Votes","Currency","Address","Locality","City","Locality Verbose","Longitude","Latitude","Country Code","Switch to order menu"],axis=1)
Selected_features_table

    

Restaurant ID           Restaurant Name  \
0           6317637          Le Petit Souffle   
1           6304287          Izakaya Kikufuji   
2           6300002    Heat - Edsa Shangri-La   
3           6318506                      Ooma   
4           6314302               Sambo Kojin   
...             ...                       ...   
9546        5915730               Naml۱ Gurme   
9547        5908749              Ceviz A��ac۱   
9548        5915807                     Huqqa   
9549        5916112               A���k Kahve   
9550        5927402  Walter's Coffee Roastery   

                              Cuisines  Average Cost for two  \
0           French, Japanese, Desserts                  1100   
1                             Japanese                  1200   
2     Seafood, Asian, Filipino, Indian                  4000   
3                      Japanese, Sushi                  1500   
4                     Japanese, Korean                  1500   
...                                ...                   ...   
9546                           Turkish                    80   
9547   World Cuisine, Patisserie, Cafe                   105   
9548            Italian, World Cuisine                   170   
9549                   Restaurant Cafe                   120   
9550                              Cafe                    55   

     Has Table booking Has Online delivery Is delivering now  Price range  \
0                  Yes                  No                No            3   
1                  Yes                  No                No            3   
2                  Yes                  No                No            4   
3                   No                  No                No            4   
4                  Yes                  No                No            4   
...                ...                 ...               ...          ...   
9546                No                  No                No            3   
9547                No                  No                No            3   
9548                No                  No                No            4   
9549                No                  No                No            4   
9550                No                  No                No            2   

      Aggregate rating  
0                  4.8  
1                  4.5  
2                  4.4  
3                  4.9  
4                  4.8  
...                ...  
9546               4.1  
9547               4.2  
9548               3.7  
9549               4.0  
9550               4.0  

[9551 rows x 9 columns]

In [14]:
Selected_features_table.columns

Index(['Restaurant ID', 'Restaurant Name', 'Cuisines', 'Average Cost for two',
       'Has Table booking', 'Has Online delivery', 'Is delivering now',
       'Price range', 'Aggregate rating'],
      dtype='object')

TO CONVERT CATEGORICAL DATA INTO NUMERICAL DATA(ENCODING CATEGORICAL VARIABLES)

In [15]:
cols=["Restaurant Name","Cuisines","Average Cost for two","Has Table booking","Has Online delivery","Is delivering now"]
le=LabelEncoder()
for i in cols:
  Selected_features_table[i]=le.fit_transform(Selected_features_table[i])
Selected_features_table

Restaurant ID  Restaurant Name  Cuisines  Average Cost for two  \
0           6317637             3748       920                    73   
1           6304287             3172      1111                    75   
2           6300002             2896      1671                   113   
3           6318506             4707      1126                    81   
4           6314302             5523      1122                    81   
...             ...              ...       ...                   ...   
9546        5915730             4443      1813                    16   
9547        5908749             1310      1824                    21   
9548        5915807             3068      1110                    29   
9549        5916112              512      1657                    23   
9550        5927402             7240       331                    11   

      Has Table booking  Has Online delivery  Is delivering now  Price range  \
0                     1                    0                  0            3   
1                     1                    0                  0            3   
2                     1                    0                  0            4   
3                     0                    0                  0            4   
4                     1                    0                  0            4   
...                 ...                  ...                ...          ...   
9546                  0                    0                  0            3   
9547                  0                    0                  0            3   
9548                  0                    0                  0            4   
9549                  0                    0                  0            4   
9550                  0                    0                  0            2   

      Aggregate rating  
0                  4.8  
1                  4.5  
2                  4.4  
3                  4.9  
4                  4.8  
...                ...  
9546               4.1  
9547               4.2  
9548               3.7  
9549               4.0  
9550               4.0  

[9551 rows x 9 columns]

GETTING THE SIMILIARITIES FOR RECOMMENDTION SYSTEM

In [16]:
similarity_1=cosine_similarity(Selected_features_table)
similarity_1

array([[1.        , 1.        , 0.99999998, ..., 1.        , 0.99999986,
        0.9999998 ],
       [1.        , 1.        , 1.        , ..., 1.        , 0.99999991,
        0.99999973],
       [0.99999998, 1.        , 1.        , ..., 1.        , 0.99999993,
        0.99999969],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 0.9999999 ,
        0.99999974],
       [0.99999986, 0.99999991, 0.99999993, ..., 0.9999999 , 1.        ,
        0.99999933],
       [0.9999998 , 0.99999973, 0.99999969, ..., 0.99999974, 0.99999933,
        1.        ]], shape=(9551, 9551))

In [ ]:
print(similarity_1.shape)

(9551, 9551)


LET US NOW CREATE A RESTAURANT RECOMMENDATION SYSTEM BASED ON USERS PREFERENCE OF RESTAURANT NAMES 

In [18]:
restaurant_name=input("Enter a name ")
restaurant_name 

'La Petit'

In [19]:
list_of_all_restaurants=df["Restaurant Name"].tolist()
print(list_of_all_restaurants)

['Le Petit Souffle', 'Izakaya Kikufuji', 'Heat - Edsa Shangri-La', 'Ooma', 'Sambo Kojin', 'Din Tai Fung', 'Buffet 101', 'Vikings', 'Spiral - Sofitel Philippine Plaza Manila', 'Locavore', 'Silantro Fil-Mex', "Mad Mark's Creamery & Good Eats", 'Silantro Fil-Mex', "Guevarra's", 'Sodam Korean Restaurant', 'Cafe Arabelle', "Nonna's Pasta & Pizzeria", 'Balay Dako', 'Hobing Korean Dessert Cafe', 'Wildflour Cafe + Bakery', 'NIU by Vikings', 'The Food Hall by Todd English', 'Chez Michou', 'Caf�� Daniel Briand', 'Casa do Biscoito Mineiro', 'Maori', 'Pizza �� Bessa', 'Sushi Loko', 'Beirute', 'New Koto', 'Sandubas Caf��', 'Villa Tevere', 'Rovereto', 'Buena Carne', 'Taco Pep', 'Coco Bambu', 'Tayp��', 'Outback Steakhouse', 'Manzu��', 'Coco Bambu', 'Gero', 'Brazilian American Burgers', 'Pesqueiro Eco Gourmet', 'Confeitaria Colombo', 'Bibi', 'Cervantes', 'Amir', 'TT Burger', 'Braseiro da G��vea', 'Balada Mix', 'Garota de Ipanema', 'Zaz�� Bistr�� Tropical', 'Fil�� de Ouro', 'D.O.C Ristorante', 'Sushi L

In [20]:
find_close_match=difflib.get_close_matches(restaurant_name,list_of_all_restaurants)
print(find_close_match)

['Getafix Petit', 'Pita Pit', 'Pita Pit']


In [21]:
close_match=find_close_match[0]
print(close_match)

Getafix Petit


Let us get the INDEX for that particualr restaurant Le Petit Souffle

In [22]:
restaurant_index=df[df["Restaurant Name"]== close_match]["Restaurant ID"].index[0]
print(restaurant_index)

8368


Let us get a similiarity scores for the list of all indexes

In [25]:
similarity_score=list(enumerate(similarity_1[restaurant_index]))

In [ ]:
similarity_score

[(0, np.float64(0.9999998936266314)),
 (1, np.float64(0.9999999269354323)),
 (2, np.float64(0.9999999266426062)),
 (3, np.float64(0.9999998100282164)),
 (4, np.float64(0.9999997234113678)),
 (5, np.float64(0.9999999993010016)),
 (6, np.float64(0.9999999992933882)),
 (7, np.float64(0.999999479274217)),
 (8, np.float64(0.9999996626518601)),
 (9, np.float64(0.9999998892817233)),
 (10, np.float64(0.9999996874875555)),
 (11, np.float64(0.9999998849717914)),
 (12, np.float64(0.9999996884870876)),
 (13, np.float64(0.9999999523663916)),
 (14, np.float64(0.9999996719596576)),
 (15, np.float64(0.9999999960817713)),
 (16, np.float64(0.9999999937363122)),
 (17, np.float64(0.9999999950325748)),
 (18, np.float64(0.999999999435697)),
 (19, np.float64(0.9999994872932817)),
 (20, np.float64(0.9999998215958982)),
 (21, np.float64(0.9999995916677845)),
 (22, np.float64(0.9999999937515449)),
 (23, np.float64(0.9999999991511416)),
 (24, np.float64(0.9999999981924921)),
 (25, np.float64(0.9999998900305713))

In [ ]:
len(similarity_score)

9551

SORTING THE RESTAURANTS BASED ON THEIR SIMILARITY SCORE 

In [ ]:
sorted_similarity_scores=sorted(similarity_score,key=lambda x:x[1],reverse=True)

In [ ]:
print(sorted_similarity_scores)

[(8368, np.float64(0.9999999999999998)), (6252, np.float64(0.9999999999991654)), (5535, np.float64(0.9999999999990131)), (3604, np.float64(0.999999999998944)), (918, np.float64(0.9999999999988916)), (1805, np.float64(0.9999999999988021)), (6370, np.float64(0.9999999999986109)), (8129, np.float64(0.9999999999984854)), (3572, np.float64(0.9999999999982646)), (7989, np.float64(0.9999999999982419)), (1638, np.float64(0.9999999999979404)), (4234, np.float64(0.9999999999976742)), (7243, np.float64(0.9999999999972026)), (7766, np.float64(0.9999999999970328)), (6792, np.float64(0.9999999999968159)), (1650, np.float64(0.9999999999966896)), (2250, np.float64(0.999999999996674)), (7013, np.float64(0.9999999999964834)), (1977, np.float64(0.9999999999964619)), (7783, np.float64(0.9999999999964614)), (1328, np.float64(0.9999999999958367)), (4106, np.float64(0.999999999995492)), (6322, np.float64(0.9999999999954061)), (3036, np.float64(0.9999999999951732)), (6707, np.float64(0.9999999999937945)), (47

PRINTING THE SIMILIAR RESTAURANTS BASED ON THE INDEX 

In [ ]:
print("Restaurants suggested for you :")
j=1
for i in sorted_similarity_scores:
    index=i[0]
    
    restaurant_name_from_index=df[df.index==index]["Restaurant Name"].values[0]   
    if j <30:
        print(restaurant_name_from_index) 
        j=j+1
                



Restaurants suggested for you :
Getafix Petit
Giani
Giani
Giani's
Giani's
GoGourmet
Giani
Giani
Gelato Vinto
GoGourmet
GoGourmet
Giani's
Giani
Giani
Giani
Giani's
Giani's
Giani
Giani
Giani
Good Food
Giani
Giani's
GoGourmet
Goosebumps
Goosebumps
Good Luck Cafe
Giani
Giani


Let us create a restaurant recommendation system based on cusisines 

In [ ]:
df

Restaurant ID           Restaurant Name  Country Code              City  \
0           6317637          Le Petit Souffle           162       Makati City   
1           6304287          Izakaya Kikufuji           162       Makati City   
2           6300002    Heat - Edsa Shangri-La           162  Mandaluyong City   
3           6318506                      Ooma           162  Mandaluyong City   
4           6314302               Sambo Kojin           162  Mandaluyong City   
...             ...                       ...           ...               ...   
9546        5915730               Naml۱ Gurme           208         ��stanbul   
9547        5908749              Ceviz A��ac۱           208         ��stanbul   
9548        5915807                     Huqqa           208         ��stanbul   
9549        5916112               A���k Kahve           208         ��stanbul   
9550        5927402  Walter's Coffee Roastery           208         ��stanbul   

                                                Address  \
0     Third Floor, Century City Mall, Kalayaan Avenu...   
1     Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2     Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3     Third Floor, Mega Fashion Hall, SM Megamall, O...   
4     Third Floor, Mega Atrium, SM Megamall, Ortigas...   
...                                                 ...   
9546  Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...   
9547  Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...   
9548  Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...   
9549  Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...   
9550  Cafea��a Mahallesi, Bademalt۱ Sokak, No 21/B, ...   

                                        Locality  \
0      Century City Mall, Poblacion, Makati City   
1     Little Tokyo, Legaspi Village, Makati City   
2     Edsa Shangri-La, Ortigas, Mandaluyong City   
3         SM Megamall, Ortigas, Mandaluyong City   
4         SM Megamall, Ortigas, Mandaluyong City   
...                                          ...   
9546                                    Karak�_y   
9547                                   Ko��uyolu   
9548                                 Kuru�_e��me   
9549                                 Kuru�_e��me   
9550                                        Moda   

                                       Locality Verbose   Longitude  \
0     Century City Mall, Poblacion, Makati City, Mak...  121.027535   
1     Little Tokyo, Legaspi Village, Makati City, Ma...  121.014101   
2     Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...  121.056831   
3     SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.056475   
4     SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.057508   
...                                                 ...         ...   
9546                                Karak�_y, ��stanbul   28.977392   
9547                               Ko��uyolu, ��stanbul   29.041297   
9548                             Kuru�_e��me, ��stanbul   29.034640   
9549                             Kuru�_e��me, ��stanbul   29.036019   
9550                                    Moda, ��stanbul   29.026016   

       Latitude                          Cuisines  ...          Currency  \
0     14.565443        French, Japanese, Desserts  ...  Botswana Pula(P)   
1     14.553708                          Japanese  ...  Botswana Pula(P)   
2     14.581404  Seafood, Asian, Filipino, Indian  ...  Botswana Pula(P)   
3     14.585318                   Japanese, Sushi  ...  Botswana Pula(P)   
4     14.584450                  Japanese, Korean  ...  Botswana Pula(P)   
...         ...                               ...  ...               ...   
9546  41.022793                           Turkish  ...  Turkish Lira(TL)   
9547  41.009847   World Cuisine, Patisserie, Cafe  ...  Turkish Lira(TL)   
9548  41.055817            Italian, World Cuisine  ...  Turkish Lira(TL)   
9549  41.057979                   Restaurant Cafe  ...  Turkish Lira(TL)   
9550  40.9

In [30]:
cusine_name="French, Japanese, Deserts"



In [31]:
list1=df["Cuisines"].astype(str).tolist()
print(list1)

['French, Japanese, Desserts', 'Japanese', 'Seafood, Asian, Filipino, Indian', 'Japanese, Sushi', 'Japanese, Korean', 'Chinese', 'Asian, European', 'Seafood, Filipino, Asian, European', 'European, Asian, Indian', 'Filipino', 'Filipino, Mexican', 'American, Ice Cream, Desserts', 'Filipino, Mexican', 'Filipino', 'Korean', 'Cafe, American, Italian, Filipino', 'Italian, Pizza', 'Filipino', 'Cafe, Korean, Desserts', 'Cafe, Bakery, American, Italian', 'Seafood, American, Mediterranean, Japanese', 'American, Asian, Italian, Seafood', 'Fast Food, French', 'Cafe', 'Bakery', 'Brazilian', 'Pizza', 'Japanese', 'Arabian', 'Japanese', 'Brazilian, Cafe', 'Italian', 'Pizza', 'Bar Food, Brazilian', 'Mexican, Grill', 'International', 'Peruvian, Latin American', 'American, Grill', 'Seafood', 'International', 'Italian', 'American, Burger', 'Seafood, Bar Food, Brazilian', 'Desserts, Cafe', 'Juices, Healthy Food', 'Beverages, Bar Food, Fast Food', 'Lebanese', 'Burger', 'Brazilian, Bar Food', 'Brazilian, Hea

In [32]:
find_close_match=difflib.get_close_matches(cusine_name,list1)
print(find_close_match)

['French, Japanese, Desserts', 'Korean, Japanese, Chinese, Asian', 'French, Brazilian, Beverages']


In [33]:
close_match=find_close_match[0]
print(close_match)

French, Japanese, Desserts


In [34]:
index_cusine=df[df["Cuisines"]==close_match]["Restaurant Name"].index[0]
index_cusine


np.int64(0)

In [35]:
similarity_scores=list(enumerate(similarity_1[int(index_cusine)]))
print(similarity_scores)

[(0, np.float64(0.9999999999999999)), (1, np.float64(0.9999999954716874)), (2, np.float64(0.9999999839044214)), (3, np.float64(0.9999999879627183)), (4, np.float64(0.9999999598861025)), (5, np.float64(0.9999998780335391)), (6, np.float64(0.9999998982598617)), (7, np.float64(0.9999998435825618)), (8, np.float64(0.99999993270717)), (9, np.float64(0.99999999993653)), (10, np.float64(0.999999944262337)), (11, np.float64(0.999999990523762)), (12, np.float64(0.9999999446786243)), (13, np.float64(0.9999999877903081)), (14, np.float64(0.9999999388636559)), (15, np.float64(0.9999998495958116)), (16, np.float64(0.9999999385371616)), (17, np.float64(0.9999998770207251)), (18, np.float64(0.9999998991865439)), (19, np.float64(0.9999998385318064)), (20, np.float64(0.9999999872472298)), (21, np.float64(0.9999998880933723)), (22, np.float64(0.9999999312146086)), (23, np.float64(0.9999999111051535)), (24, np.float64(0.999999914003451)), (25, np.float64(0.9999999941143521)), (26, np.float64(0.9999999822

In [36]:
sorted_similarity=sorted(similarity_scores,key=lambda x :x[1],reverse=True)
sorted_similarity

[(0, np.float64(0.9999999999999999)),
 (9, np.float64(0.99999999993653)),
 (74, np.float64(0.9999999998223087)),
 (9492, np.float64(0.9999999997502472)),
 (9544, np.float64(0.9999999996429831)),
 (9372, np.float64(0.9999999994401416)),
 (67, np.float64(0.9999999994095222)),
 (604, np.float64(0.9999999993062253)),
 (9299, np.float64(0.9999999992496492)),
 (607, np.float64(0.9999999991704074)),
 (9410, np.float64(0.9999999988062279)),
 (9415, np.float64(0.9999999987735675)),
 (9527, np.float64(0.9999999987431922)),
 (9420, np.float64(0.9999999985592941)),
 (752, np.float64(0.9999999985589523)),
 (9411, np.float64(0.9999999985419331)),
 (9444, np.float64(0.9999999983953213)),
 (9350, np.float64(0.9999999983613899)),
 (9537, np.float64(0.9999999982532608)),
 (857, np.float64(0.9999999980861323)),
 (9330, np.float64(0.9999999980421465)),
 (9331, np.float64(0.9999999979699044)),
 (9320, np.float64(0.9999999976137062)),
 (9333, np.float64(0.9999999975626631)),
 (9395, np.float64(0.99999999755

In [37]:
print("Restaurants suggested for you :")
j=1
for i in sorted_similarity:
    index=i[0]
    
    restaurant_name_from_index=df[df.index==index]["Restaurant Name"].values[0]   
    if j <30:
        print(restaurant_name_from_index) 
        j=j+1

Restaurants suggested for you :
Le Petit Souffle
Locavore
Meats
Licorish Bistro
Karak�_y G�_ll�_o��lu
Mother India's Cafe
Les 3 Brasseurs
Kamat
Milse
Katis Restaurant
Mughli
Mr Cooper's House & Garden - The Midland
Kebap 49
MRA Bakery Sweets & Restaurant
Chi Kitchen
Lahore
Kloof Street House
Lasan Restaurant
J'adore Chocolatier
Dunkin Donuts
Little Penang
Loretta
Ombra
Olive
Jamie's Italian
Hot Palayok
Coffee Culture
Masala Zone
Mainland China Restaurant
